# Offline alignment algorithms for transformer

We now take a more practical view towards offline alignment algorithms using GPT-2 models. We will initialize from a pretrained GPT-2 model, carry out an initial round of supervised fine-tuning (SFT), followed by preference optimization.

This lab is heavily built upon the open source project (nano-gpt: https://github.com/karpathy/nanoGPT).

## GSM8k

In this lab, we will focus on GSM8k (https://arxiv.org/abs/2305.20050), a dataset of about 8000 graduate school math question and answer pairs. We will build training datasets from the training split and evaluation using the test split. Make sure that you can download the dataset.

In [6]:
import json

# TODO: maybe download first to a shared directly and load into colabs, update here
TRAINING_DIR = '/Users/tangyunhao/Documents/mingpt/demo/build-nanogpt/data/train.jsonl'
TESTING_DIR = '/Users/tangyunhao/Documents/mingpt/demo/build-nanogpt/data/test.jsonl'

def load_jsonl(dir):
    with open(dir, 'r') as json_file:
        json_list = list(json_file)

    results = []
    for json_str in json_list:
        result = json.loads(json_str)
        results.append(result)
    return results

training_set = load_jsonl(TRAINING_DIR)
testing_set = load_jsonl(TESTING_DIR)

In [7]:
print(training_set[0]["question"])
print('===')
print(training_set[0]["answer"])

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
===
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


## Tokenizer

Transformers consume tokens, which are integer representations of strings. We create a tokenizer and illustrate how a string is embedded into tokens.

In [8]:
enc = tiktoken.get_encoding("gpt2")

input = "hello world, salut !"
print(enc.encode(input))


[31373, 995, 11, 3664, 315, 5145]


## Transformers

Below we provide a number of basic definitions for the transformer class, and utilities to initialize from a GPT-2 pretrained model.

In [1]:
from dataclasses import dataclass
import torch
import torch.nn as nn
import math
import numpy as np
from torch.nn import functional as F


@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768


class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.config = config
        # key, query value projection for all heads
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(
            torch.ones(self.config.block_size,
                        self.config.block_size)).view(
                            1, 1, self.config.block_size, self.config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        
        # attn materialization
        # attn = q @ k.transpose(-2, -1) * (1.0 / math.sqrt(k.size(-1)))
        # attn = attn.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        # attn = F.softmax(attn, dim=-1)
        # y = attn @ v
        # flash attention
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        
        # TODO(robintyh): understand contiguous better
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # reassemble head outputs
        # output projection
        y = self.c_proj(y)
        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        h_size = 4 * config.n_embd
        self.c_fc = nn.Linear(config.n_embd, h_size)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(h_size, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config: GPTConfig):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):

    def __init__(self, config: GPTConfig):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(wte = nn.Embedding(config.vocab_size, config.n_embd),
                                              wpe = nn.Embedding(config.block_size, config.n_embd),
                                              h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                                              ln_f = nn.LayerNorm(config.n_embd)))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing
        self.transformer.wte.weight = self.lm_head.weight

        # init params
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            # adjust initialization due to residual pathway
            if hasattr(module, "NANOGPT_SCALE_INIT"):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def forward(self, idx, targets=None, masks=None):
        # idx (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size {self.config.block_size}"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        token_emb = self.transformer.wte(idx)
        x = token_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        if targets is None:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None
        else:
            logits = self.lm_head(x)  # (B, T, vocab_size)
            # assert masks is not None
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), reduction='none')
            loss = torch.sum(loss.view(B, T) * masks) / (1e-8 + torch.sum(masks))
            # loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        return logits, loss

## Sampling

A key element is to sample from the transformer model, which requires auto-regressive generation. We can make it more efficient in general but here stick with proof-of-concept implementation.

In [9]:
def sample_from_model(model, input, top_k=50, num_return_sequences=4, max_length=128):
    tokens = enc.encode(input)
    tokens = torch.tensor(tokens, dtype=torch.long)
    tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
    x = tokens.to("mps")
    prompt_length = x.size(1)

    while x.size(1) < max_length:
        with torch.no_grad():
            logits, _ = model(x)  # (B, T, vocab_size)
            logits = logits[:, -1, :]  # take last position
            probs = F.softmax(logits, dim=-1)
            # top-k sampling
            topk_probs, topk_indices = torch.topk(probs, top_k, dim=-1)
            ix = torch.multinomial(topk_probs, 1)
            xcol = torch.gather(topk_indices, -1, ix)
            # append to the sequence
            x = torch.cat([x, xcol], axis=1)
    x = x[:, prompt_length:]
    all_decoded = []
    for i in range(num_return_sequences):
        tokens = x[i, :max_length].tolist()
        all_decoded.append(enc.decode(tokens))
    return all_decoded

### Evaluation

We now build an evaluation based on the test set, to better monitor the model's training progress over time.

In [24]:
def format_input(question):
    return "Question: " + question + "\n\nAnswer:"


def format_output(answer):
    return answer + "<EOS>"


def evaluate(model, num_examples=10, num_return_sequences=4):
    accs_all = []
    for example in testing_set[:num_examples]:
        input = format_input(example["question"])
        generations = sample_from_model(model, input=input, num_return_sequences=num_return_sequences)
        accs = []
        for g in generations:
            answer = g.split("####")[-1].strip()
            if answer.lower() == example["answer"].lower():
                accs.append(1.0)
            else:
                accs.append(0.0)
        accs_all.append(accs)
    accs_all = np.array(accs_all)
    return np.mean(np.max(accs_all, axis=-1))

### Dataloader for SFT

A useful abstract here is a dataloader. A dataloader loads data and tokenizes them for training beforehand. It also handles padding and masking, making sure that the data to be consumed is batched properly.

In [3]:
import tiktoken


class DataLoaderLite():
    def __init__(self, B, T, inputs, outputs):
        assert len(inputs) == len(outputs)
        print(f"loaded {len(inputs)} sequences")
        self.B = B
        self.T = T
        self.tokens = []
        self.masks = []

        enc = tiktoken.get_encoding("gpt2")
        num_padding = 0
        num_truncation = 0
        for input, output in zip(inputs, outputs):
            input_tokens = enc.encode(input)
            output_tokens = enc.encode(output)
            tokens = input_tokens + output_tokens
            # padding tokens
            masks = [False] * len(input_tokens) + [True] * len(output_tokens)
            if len(tokens) <= T + 1:
                num_padding += (T + 1 - len(tokens))
                masks = masks + [False] * (T + 1- len(tokens))
                tokens = tokens + [0] * (T + 1- len(tokens))
            else:
                num_truncation += 1
                tokens = tokens[:T + 1]
                masks = masks[:T + 1]
            assert len(masks) == len(tokens) == T + 1
            self.tokens.append(tokens)
            self.masks.append(masks)
        self.tokens = torch.tensor(self.tokens, dtype=int)
        self.masks = torch.tensor(self.masks, dtype=bool)
        print(f"proportion of truncated sequences {num_truncation / len(self.tokens)}")
        print(f"proportion of padded tokens {num_padding / self.tokens.numel()}")
        print(f"loaded {self.tokens.numel()} tokens")

        # state
        self.current_pos = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_pos : self.current_pos + B]
        mask_buf = self.masks[self.current_pos : self.current_pos + B]
        x = buf[:, :-1]
        y = buf[:, 1:]
        mask = mask_buf[:, 1:]
        self.current_pos += B
        if self.current_pos + B + 1 > len(self.tokens):
            self.current_pos = 0
        return x, y, mask

In [11]:
inputs = [format_input(train["question"]) for train in training_set]
outputs = [train["answer"] for train in training_set]
dataloader = DataLoaderLite(B=2, T=256, inputs=inputs, outputs=outputs)
x, y, mask = dataloader.next_batch()

loaded 7473 sequences
proportion of truncated sequences 0.05687140372005888
proportion of padded tokens 0.40036999605844337
loaded 1920561 tokens


In [53]:
def generate_random_examples():
    a = np.random.randint(100)
    b = np.random.randint(100)
    return f"What is {a} + {b}?", f" {a + b}"

qas = [generate_random_examples() for _ in range(1000)]
qas_test = [generate_random_examples() for _ in range(100)]
inputs = [format_input(qa[0]) for qa in qas]
outputs = [format_output(qa[1]) for qa in qas]
inputs_test = [format_input(qa[0]) for qa in qas_test]
outputs_test = [format_output(qa[1]) for qa in qas_test]

In [26]:
inputs[0], outputs[0]

('Question: What is 23 + 36?\n\nAnswer:', ' 59<EOS>')

In [52]:
# main loop
import time

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")


#inputs = [format_input(train["question"]) for train in training_set]
#outputs = [train["answer"] for train in training_set]
#inputs = inputs[:4]
#outputs = outputs[:4]
train_loader = DataLoaderLite(B=4, T=32, inputs=inputs, outputs=outputs)
torch.set_float32_matmul_precision("high")

# optimization loop
torch.manual_seed(42)

# n_layer=24, n_head=16, n_embd=1024
model = GPT(GPTConfig())
model.from_pretrained("gpt2")
model.to(device)
# model = torch.compile(model)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
for i in range(1000):
    t0 = time.time()
    x, y, mask = train_loader.next_batch()
    x, y, mask = x.to(device), y.to(device), mask.to(device)
    optimizer.zero_grad()
    # with torch.autocast(device_type=device, dtype=torch.bfloat16):
    logits, loss = model(x, y, mask)
    loss.backward()
    optimizer.step()
    torch.mps.synchronize()
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    if i % 100 == 0:
        eval_train_res = evaluate(model, inputs, outputs)
        eval_test_res = evaluate(model, inputs_test, outputs_test)
        print(f"step {i}, loss {loss.item()}, eval train {eval:.2f} dt {dt:.2f}ms, tokens per sec {tokens_per_sec}")

using device: mps
loaded 100 sequences
proportion of truncated sequences 0.0
proportion of padded tokens 0.48484848484848486
loaded 3300 tokens
loading weights from pretrained gpt: gpt2
step 0, loss 10.716630935668945, eval 0.00 dt 227.17ms, tokens per sec 563.4464261350876
step 100, loss 0.9409936666488647, eval 0.20 dt 175.44ms, tokens per sec 729.6067493300137
step 200, loss 0.9402273297309875, eval 0.20 dt 175.55ms, tokens per sec 729.1241914214278
step 300, loss 0.846878707408905, eval 0.00 dt 175.63ms, tokens per sec 728.8005321387361
step 400, loss 0.7246896028518677, eval 0.20 dt 176.21ms, tokens per sec 726.4013811709916
step 500, loss 0.6450914740562439, eval 0.30 dt 176.75ms, tokens per sec 724.1791162350897
step 600, loss 0.6302787065505981, eval 0.50 dt 190.13ms, tokens per sec 673.2092491347745
step 700, loss 0.45431023836135864, eval 0.80 dt 253.56ms, tokens per sec 504.80992862321125
step 800, loss 0.6805833578109741, eval 0.60 dt 318.66ms, tokens per sec 401.6781103870

In [28]:
enc.decode(y[0].tolist()), mask[0]

(': What is 13 + 79?\n\nAnswer: 92<EOS>!!!!!!!!!!!!!!!!',
 tensor([False, False, False, False, False, False, False, False, False, False,
         False,  True,  True,  True,  True,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False], device='mps:0'))

In [23]:
enc.decode(y[0][mask[0]].tolist())

' 178'

In [50]:
def eval_train(model):
    all_accs = []
    for input, output in zip(inputs[:10], outputs[:10]):
        samples = sample_from_model(model, input=input, top_k=10, num_return_sequences=4, max_length=20)
        accs = []
        for s in samples:
            answer = s.split("<EOS>")[0].lower().strip()
            gt_answer = output.split("<EOS>")[0].lower().strip()
            if answer == gt_answer:
                accs.append(1.0)
            else:
                accs.append(0.0)
        all_accs.append(np.max(accs))
    return np.mean(all_accs)

eval_train(model)

0.1

In [48]:
samples, inputs[2]

([' 47<EOS><EOS', ' 64<EOS><EOS', ' 81<EOS><EOS', ' 179<EOS><EOS'],
 'Question: What is 14 + 69?\n\nAnswer:')

In [42]:
14 + 69

83

In [76]:
testing_set[0]["question"]

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [79]:
print(enc.decode(list(x[0])))

A question and answer forum has 200 members. The average number of answers posted by each member on the forum is three times as many as the number of questions asked. If each user posts an average of 3 questions per hour, calculate the total number of questions and answers posted on the forum by its users in a day.If each user posts an average of 3 questions per hour, the average number of answers posted on the forum is 3 times the number of questions which totals to 3*3 = <<3*3=9>>9 answers per
hour.
In a day, with 24 hours, the average number of questions asked by each member is 24*3 = <<24*3=72>>72,
If there are 200 users of the forum, the total number of questions asked is 200*72 = <<200*72=14400>>14400 questions in a day.
At the same time, a member posts an average of 9*24 = <<9*24=216>>216 answers in a day.
Since there are 200 members who use the forum, the total number of answers posted in the forum in a day is 200*216 = <<200*216=43200>>43200.
In total, the number of questions


In [84]:
i = 0
print(inputs[i])
print(outputs[i])

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


In [86]:
print(enc.decode(list(x[1])))
print(enc.decode(list(y[1])))

Mark and his sister Chris both leave their house for school at the same time.  Mark travels at the same speed as Chris, who walks 3 miles per hour. After walking 3 miles Mark has to turn around and go home because he forgot his lunch. If the distance from their house to the school is 9 miles, how much longer does Mark spend walking than Chris?First find the total number of miles Mark walks to get his lunch by multiplying the distance he walks back to the house by 2: 3 miles * 2 = <<3*2=6>>6 miles
Then find the total distance Mark walks by adding that distance to the distance between the house and the school: 6 miles + 9 miles = <<6+9=15>>15 miles
Now divide the total distance Mark walks by his speed to find how many hours he spends walking: 15 miles / 3 miles/hour = <<15/3=5>>5 hours
Now divide the total distance Chris walks by her speed to find the total time she spends walking: 9 miles / 3 miles/hour = <<9/3=3>>3 hours
Now subtract the time Chris spends walking from the time Mark spe

In [2]:
# data loader
import tiktoken
class OldDataLoaderLite():
    def __init__(self, B, T):
        self.B = B
        self.T = T

        enc = tiktoken.get_encoding("gpt2")
        with open('input.txt', 'r') as f:
            text = f.read()
        data = text[:1000]
        self.tokens =torch.tensor(enc.encode(text))
        print(f"loaded {len(self.tokens)} tokens")
        print(f"epoch = {len(self.tokens) // (B * T)} batches")

        # state
        self.current_pos = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_pos : self.current_pos + B * T + 1]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)
        self.current_pos += B * T
        if self.current_pos + (B * T + 1) > len(self.tokens):
            self.current_pos = 0
        return x, y


# main loop
import time

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")


train_loader = OldDataLoaderLite(B=2, T=1024)
torch.set_float32_matmul_precision("high")

# optimization loop
torch.manual_seed(42)

model = GPT(GPTConfig())
model.to(device)
# model = torch.compile(model)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
for i in range(50):
    t0 = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    mask = torch.ones_like(y).to(device)
    optimizer.zero_grad()
    # with torch.autocast(device_type=device, dtype=torch.bfloat16):
    logits, loss = model(x, y,mask)
    loss.backward()
    optimizer.step()
    torch.mps.synchronize()
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f"step {i}, loss {loss.item()}, dt {dt:.2f}ms, tokens per sec {tokens_per_sec}")


using device: mps
loaded 338025 tokens
epoch = 165 batches
step 0, loss 11.017706871032715, dt 1418.90ms, tokens per sec 1443.3684598233901
step 1, loss 9.563642501831055, dt 1209.63ms, tokens per sec 1693.083689034726
step 2, loss 9.22248363494873, dt 1193.94ms, tokens per sec 1715.3322701786715
step 3, loss 8.806707382202148, dt 1199.08ms, tokens per sec 1707.968997044734
step 4, loss 8.736193656921387, dt 1195.19ms, tokens per sec 1713.5337867948774
step 5, loss 8.490167617797852, dt 1193.73ms, tokens per sec 1715.635125769595
step 6, loss 8.184244155883789, dt 1193.45ms, tokens per sec 1716.032014728821
step 7, loss 8.013078689575195, dt 1192.58ms, tokens per sec 1717.281115861381
step 8, loss 7.741802215576172, dt 1197.15ms, tokens per sec 1710.7252319354802
step 9, loss 7.522797584533691, dt 1194.84ms, tokens per sec 1714.0326450877824
step 10, loss 7.360325813293457, dt 1192.41ms, tokens per sec 1717.5228437276248
step 11, loss 6.869381904602051, dt 1205.52ms, tokens per sec 169

KeyboardInterrupt: 